In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import random
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import datasets, transforms
import torchtext

def visualize_mnist(dataloader, model=None, title=""):
    x, y = next(iter(dataloader_test))
    fig, axes = plt.subplots(2, 5, layout="constrained")
    for i in range(10):
        ax = axes[i//5, i%5]
        for j in range(BATCH_SIZE):
            if y[j].item() == i:
                break
#         plt.subplot(2, 5, i+1)
        ax.imshow(x[j, 0], cmap='gray')
        if model is not None:
            y_hat = model(x[j:j+1].cuda()).argmax()
            ax.set_title(f"Prediction: {y_hat.item()}")
        else:
            ax.set_title(f"Answer: {y[j].item()}")
        ax.axis('off')
    fig.tight_layout(pad=0.2, h_pad=-5)
    fig.suptitle(title, y=0.93)
    plt.show()

# Lab3-1. Image Classification Using FC Layers

In [ ]:
BATCH_SIZE = 500
LR = 0.001
TOTAL_EPOCH = 10

# Prepare MNIST datasets
transform = transforms.Compose([
    transforms.ToTensor(),                        # 입력을 PyTorch tensor로 바꾸겠다
    transforms.Normalize((0.1307,), (0.3081,))    # mean=0.1307, std=0.3081 로 normalize 하겠다
])
dataset_train = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset_test = datasets.MNIST('../data', train=False, transform=transform)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE)

# Visualize
visualize_mnist(dataloader_test, title="MNIST")

In [ ]:
# Define a network
class Net(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(28*28, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        # x: [Batch_size, Channels, Height, Width] = [500, 1, 28, 28]
        x = x.reshape(-1, 28*28) # x.shape: [Batch_size, 768]
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x


model = Net(1024).cuda()  # Create a network (parameters are randomly initialized)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)  # Create an optimizer


# Visualize model predictions before training
visualize_mnist(dataloader_test, model, "Model prediction before training")

In [ ]:
def train(model, dataloader, optimizer, epoch):
    model.train()
    train_loss = 0.0
    iteration = 0
    for x, target in dataloader:
        iteration = iteration + 1
        x, target = x.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(x)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        if iteration % 10 == 0:
            print(
                f'\r[Training...] Epoch: {epoch}, '
                f'Iteration: {iteration}/{len(dataloader)}, '
                f'train_loss: {train_loss / iteration:.4f}',
                flush=True,
                end=""
            )
    return train_loss / iteration


@torch.no_grad()
def test(model, dataloader):
    model.eval()
    test_loss = 0
    correct = 0
    iteration = 0
    for x, target in dataloader:
        iteration = iteration + 1
        x, target = x.cuda(), target.cuda()
        output = model(x)
        test_loss += F.cross_entropy(output, target).item()
        pred = output.argmax(dim=1)  # get the index of the max log-probability
        correct += (pred == target).sum().item()
    test_loss = test_loss / len(dataloader)
    accuracy = 100 * correct / len(dataloader.dataset)
    return test_loss, accuracy


# 학습 전의 loss 측정
test_loss, test_accuracy = test(model, dataloader_test)
print(f"\rBefore Training - "
      f"test_loss: {test_loss:.4f}, test_accuracy: {test_accuracy:.1f}%")


# 학습 하면서 loss 측정
for epoch in range(1, TOTAL_EPOCH+1):
    train_loss = train(model, dataloader_train, optimizer, epoch)
    test_loss, test_accuracy = test(model, dataloader_test)
    print(f"\rEpoch {epoch} - train_loss: {train_loss:.4f}, "
          f"test_loss: {test_loss:.4f}, test_accuracy: {test_accuracy:.1f}%")


# Visualize model predictions after training
visualize_mnist(dataloader_test, model, "Model prediction after training")

# Lab3-2. Image Classification Using CNN

In [ ]:
BATCH_SIZE = 500
LR = 0.001
TOTAL_EPOCH = 10


# Prepare MNIST datasets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset_train = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset_test = datasets.MNIST('../data', train=False, transform=transform)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE)


# Define a network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,  out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.avgpool = nn.AvgPool2d(2)
        self.fc = nn.Linear(128*3*3, 10)

    def forward(self, x):
        # x.shape = [Batch_size, 1, 28, 28]
        x = self.conv1(x)       # x.shape = [BATCH_SIZE, 32, 28, 28]
        x = F.relu(x)
        x = self.avgpool(x)     # x.shape = [BATCH_SIZE, 32, 14, 14]
        x = self.conv2(x)       # x.shape = [BATCH_SIZE, 64, 14, 14]
        x = F.relu(x)
        x = self.avgpool(x)     # x.shape = [BATCH_SIZE, 64, 7, 7]
        x = self.conv3(x)       # x.shape = [BATCH_SIZE, 128, 7, 7]
        x = F.relu(x)
        x = self.avgpool(x)     # x.shape = [BATCH_SIZE, 128, 3, 3]
        x = torch.flatten(x, 1) # x.shape = [BATCH_SIZE, 128*3*3]
        x = self.fc(x)         # x.shape = [BATCH_SIZE, 10]
        # output = F.softmax(x, dim=1)
        return x


model = Net().cuda()  # Create a network (parameters are randomly initialized)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)  # Create an optimizer


# Visualize model predictions before training
visualize_mnist(dataloader_test, model, "Model prediction before training")

In [ ]:
# Train 하기 전의 loss 측정
test_loss, test_accuracy = test(model, dataloader_test)
print(f"\rBefore Training - "
      f"test_loss: {test_loss:.4f}, test_accuracy: {test_accuracy:.1f}%")


# Train & validate 하면서 loss 측정
for epoch in range(1, TOTAL_EPOCH+1):
    train_loss = train(model, dataloader_train, optimizer, epoch)
    test_loss, test_accuracy = test(model, dataloader_test)
    print(f"\rEpoch {epoch} - train_loss: {train_loss:.4f}, "
          f"test_loss: {test_loss:.4f}, test_accuracy: {test_accuracy:.1f}%")


# Visualize model predictions after training
visualize_mnist(dataloader_test, model, "Model prediction after training")

# Lab3-3. Speech Command Classification Using RNN

In [ ]:
# Prepare datasets
class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("./", download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.normpath(os.path.join(self._path, line.strip())) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]


train_set = SubsetSC("training")
test_set = SubsetSC("testing")

waveform, sample_rate, label, speaker_id, utterance_number = train_set[0]

In [ ]:
import IPython.display as ipd

waveform, sample_rate, label, speaker_id, utterance_number = train_set[1]
plt.plot(waveform.t().numpy())
plt.show()
ipd.display(ipd.Audio(waveform.numpy(), rate=sample_rate))
print(label)

In [ ]:
labels = sorted(list(set(datapoint[2] for datapoint in train_set)))

def label_to_index(word):
    # Return the position of the word in labels
    return torch.tensor(labels.index(word))


def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return labels[index]

In [ ]:
BATCH_SIZE = 256


def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)


def collate_fn(batch):
    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number
    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for waveform, _, label, *_ in batch:
        tensors += [waveform]
        targets += [label_to_index(label)]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
)

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size=80, hidden_size=384, num_layers=4, output_size=35):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.1,
            bidirectional=True,
        )
        self.output = nn.Linear(2*hidden_size, output_size)

    def forward(self, x):
        # x.shape: [BATCH_SIZE, 1, 80, Length]
        x = x.squeeze(1).transpose(1, 2)  # x.shape: [BATCH_SIZE, LENGTH, 80]
        _, (h, c) = self.lstm(x)          # h.shape: [2*num_layers, BATCH_SIZE, hidden_size]
        x = h.transpose(0, 1)[:, -2:, :].reshape(-1, 2*self.hidden_size)  # [BATCH_SIZE, 2*hidden_size]
        x = self.output(x)
        return x
        

model = Net().cuda()
mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate, n_fft=512, hop_length=256, n_mels=80).cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)


def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (x, target) in enumerate(train_loader):
        x = x.cuda()
        target = target.cuda()

        # forward propagation
        x = mel_spectrogram(x)
        output = model(x)

        # loss
        loss = F.cross_entropy(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        print(
            f"\rTrain Epoch: {epoch} [{batch_idx * len(x)}/{len(train_loader.dataset)} "
            f"({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}",
            flush=True,
            end=""
        )

        # record loss
        losses.append(loss.item())


def test(model, epoch):
    model.eval()
    correct = 0
    for x, target in test_loader:
        x = x.cuda()
        target = target.cuda()

        # forward propagation
        x = mel_spectrogram(x)
        output = model(x)

        pred = output.argmax(dim=-1)
        correct += (pred == target).sum().item()

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")


TOTAL_EPOCH = 2
losses = []

# The transform needs to live on the same device as the model and the data.
for epoch in range(1, TOTAL_EPOCH + 1):
    train(model, epoch, log_interval)
    test(model, epoch)